In [1]:
import pandas as pd
import math
import string
from collections import Counter
import re
import numpy as np

In [2]:
bs_train = pd.read_csv("./Dataset/BS_FakeNews/bs_train.csv") #Read Data

In [3]:
bs_bias = bs_train[bs_train["type"] == 0]
bs_cons = bs_train[bs_train["type"] == 1]
bs_fake = bs_train[bs_train["type"] == 2]
bs_hate = bs_train[bs_train["type"] == 3]
bs_jsci = bs_train[bs_train["type"] == 4]
bs_sati = bs_train[bs_train["type"] == 5]
bs_stat = bs_train[bs_train["type"] == 6] #分類

In [4]:
bs_bias["text"] = bs_bias["text"].fillna(" ")  #空文章填滿(為什麼有空文章啊)

/home/alfred/anaconda3/envs/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
tf_bias = dict()  #建立文字->tf分數的dictionary
tf_cons = dict()
tf_fake = dict()
tf_hate = dict()
tf_jsci = dict()
tf_sati = dict()
tf_stat = dict()


idf = dict()  #文字->idf分數(文章之間的出現頻率)

In [7]:
#計算TF
for words in bs_bias["text"]:   #讀每篇文章
    if pd.isnull(words): continue   #文章是空的就算了
    word_found = set()          #用set紀錄找到的文字
    for word in words.split():  #文章拆成單節
        try:
            tf_bias[word] += 1  #bias類別中的文章中的單字次數增加
        except:
            tf_bias[word] = tf_bias.get(word , 1) 
                #其餘新增
            tf_cons[word] = tf_cons.get(word , 0)
            tf_fake[word] = tf_fake.get(word , 0)
            tf_hate[word] = tf_hate.get(word , 0)
            tf_jsci[word] = tf_jsci.get(word , 0)
            tf_sati[word] = tf_sati.get(word , 0)
            tf_stat[word] = tf_stat.get(word , 0)
        if not(word in word_found):
            try:
                idf[word] += 1
            except:
                idf[word] = idf.get(word , 1)
        word_found.add(word)

        
for words in bs_cons["text"]:
    if pd.isnull(words): continue
    word_found = set()
    for word in words.split():
        try:
            tf_cons[word] += 1
        except:
            tf_cons[word] = tf_bias.get(word , 1)
            #其餘新增
            tf_bias[word] = tf_cons.get(word , 0)
            tf_fake[word] = tf_fake.get(word , 0)
            tf_hate[word] = tf_hate.get(word , 0)
            tf_jsci[word] = tf_jsci.get(word , 0)
            tf_sati[word] = tf_sati.get(word , 0)
            tf_stat[word] = tf_stat.get(word , 0)

        if not(word in word_found):
            try:
                idf[word] += 1
            except:
                idf[word] = idf.get(word , 1)
        word_found.add(word)

for words in bs_fake["text"]:
    if pd.isnull(words): continue
    word_found = set()
    for word in words.split():
        try:
            tf_fake[word] += 1
        except:#沒在dictionary找到的話
            tf_fake[word] = tf_fake.get(word , 1)
            #其餘新增
            tf_bias[word] = tf_bias.get(word , 0)
            tf_cons[word] = tf_cons.get(word , 0)
            tf_hate[word] = tf_hate.get(word , 0)
            tf_jsci[word] = tf_jsci.get(word , 0)
            tf_sati[word] = tf_sati.get(word , 0)
            tf_stat[word] = tf_stat.get(word , 0)   
        if not(word in word_found):#在集合中找到的話
            try:
                idf[word] += 1 #idf+1
            except:
                idf[word] = idf.get(word , 1)#新增在集合理
        word_found.add(word)
        
for words in bs_hate["text"]:
    if pd.isnull(words): continue
    word_found = set()
    for word in words.split():
        try:
            tf_hate[word] += 1
        except:
            tf_hate[word] = tf_hate.get(word , 1)
            tf_bias[word] = tf_bias.get(word , 0)
            tf_cons[word] = tf_cons.get(word , 0)
            tf_fake[word] = tf_fake.get(word , 0)
            tf_jsci[word] = tf_jsci.get(word , 0)
            tf_sati[word] = tf_sati.get(word , 0)
            tf_stat[word] = tf_stat.get(word , 0)    
        if not(word in word_found):
            try:
                idf[word] += 1
            except:
                idf[word] = idf.get(word , 1)
        word_found.add(word)
        
for words in bs_jsci["text"]:
    if pd.isnull(words): continue
    word_found = set()
    for word in words.split():
        try:
            tf_jsci[word] += 1
        except:
            tf_jsci[word] = tf_jsci.get(word , 1)
            tf_bias[word] = tf_bias.get(word , 0)
            tf_cons[word] = tf_cons.get(word , 0)
            tf_fake[word] = tf_fake.get(word , 0)
            tf_hate[word] = tf_hate.get(word , 0)
            tf_sati[word] = tf_sati.get(word , 0)
            tf_stat[word] = tf_stat.get(word , 0)   
        if not(word in word_found):
            try:
                idf[word] += 1
            except:
                idf[word] = idf.get(word , 1)
        word_found.add(word)
        
for words in bs_sati["text"]:
    if pd.isnull(words): continue
    word_found = set()
    for word in words.split():
        try:
            tf_sati[word] += 1
        except:
            tf_sati[word] = tf_sati.get(word , 1)
            #其餘新增        
            tf_bias[word] = tf_bias.get(word , 0)
            tf_cons[word] = tf_cons.get(word , 0)
            tf_fake[word] = tf_fake.get(word , 0)
            tf_jsci[word] = tf_jsci.get(word , 0)
            tf_sati[word] = tf_sati.get(word , 0)
            tf_stat[word] = tf_stat.get(word , 0)   
        if not(word in word_found):
            try:
                idf[word] += 1
            except:
                idf[word] = idf.get(word , 1)
        word_found.add(word)
    
for words in bs_stat["text"]:
    if pd.isnull(words): continue
    word_found = set()
    for word in words.split():
        try:
            tf_stat[word] += 1
        except:
            tf_bias[word] = tf_bias.get(word , 0)
            tf_cons[word] = tf_cons.get(word , 0)
            tf_fake[word] = tf_fake.get(word , 0)
            tf_hate[word] = tf_hate.get(word , 0)
            tf_jsci[word] = tf_jsci.get(word , 0)
            tf_sati[word] = tf_sati.get(word , 0)
        if not(word in word_found):
            try:
                idf[word] += 1
            except:
                idf[word] = idf.get(word , 1)
        word_found.add(word)
    

In [18]:
tf_idf_df = pd.DataFrame(list(zip(tf_bias.keys() , tf_bias.values() , tf_cons.values() , tf_fake.values() , tf_hate.values() , tf_jsci.values() , tf_sati.values() , tf_stat.values() , idf.values())))

In [19]:
tf_idf_df.columns = ["word" , "bias" , "cons" , "fake" , "hate" , "jsci" , "sati" , "stat" , "IDF"]

In [20]:
tf_idf_df

,word,bias,cons,fake,hate,jsci,sati,stat,IDF
0,print,216,36,0,10,2,4,0,205
1,pay,90,100,5,63,15,8,9,194
2,back,254,238,10,186,69,48,32,491
3,money,152,142,0,130,24,7,11,245
4,plu,16,36,0,12,13,1,2,55
5,interest,78,166,3,124,12,5,5,269
6,entir,92,64,0,55,22,15,2,198
7,famili,150,172,1,108,31,24,23,270
8,everyon,76,60,2,66,10,16,2,168
9,came,114,80,2,65,12,13,19,256


In [21]:
#計算tf-idf

#tf算法:該領域單字總出現次數/文章數
#idf:log(所有文章總數/文章中該單字出現總次數)
#兩者相乘得tf-idf

tf_idf_df["bias"] = tf_idf_df["bias"].astype("float") / float(bs_bias["text"].size)
tf_idf_df["cons"] = tf_idf_df["cons"].astype("float") / float(bs_cons["text"].size)
tf_idf_df["fake"] = tf_idf_df["fake"].astype("float") / float(bs_fake["text"].size)
tf_idf_df["hate"] = tf_idf_df["hate"].astype("float") / float(bs_hate["text"].size)
tf_idf_df["jsci"] = tf_idf_df["jsci"].astype("float") / float(bs_jsci["text"].size)
tf_idf_df["sati"] = tf_idf_df["sati"].astype("float") / float(bs_sati["text"].size)
tf_idf_df["stat"] = tf_idf_df["stat"].astype("float") / float(bs_stat["text"].size)
tf_idf_df["IDF"]  = np.log10(float(bs_train.shape[0]) / tf_idf_df["IDF"].astype("float"))

In [22]:
tf_idf_df

,word,bias,cons,fake,hate,jsci,sati,stat,IDF
0,print,0.487585,0.083721,0.000000,0.040650,0.019608,0.027397,0.000000,0.866359
1,pay,0.203160,0.232558,0.263158,0.256098,0.147059,0.054795,0.074380,0.890312
2,back,0.573363,0.553488,0.526316,0.756098,0.676471,0.328767,0.264463,0.487032
3,money,0.343115,0.330233,0.000000,0.528455,0.235294,0.047945,0.090909,0.788947
4,plu,0.036117,0.083721,0.000000,0.048780,0.127451,0.006849,0.016529,1.437751
5,interest,0.176072,0.386047,0.157895,0.504065,0.117647,0.034247,0.041322,0.748361
6,entir,0.207675,0.148837,0.000000,0.223577,0.215686,0.102740,0.016529,0.881448
7,famili,0.338600,0.400000,0.052632,0.439024,0.303922,0.164384,0.190083,0.746749
8,everyon,0.171558,0.139535,0.105263,0.268293,0.098039,0.109589,0.016529,0.952804
9,came,0.257336,0.186047,0.105263,0.264228,0.117647,0.089041,0.157025,0.769873


In [23]:
tf_idf_result = tf_idf_df

In [24]:
tf_idf_result["bias"] = tf_idf_df["bias"] * tf_idf_df["IDF"]
tf_idf_result["cons"] = tf_idf_df["cons"] * tf_idf_df["IDF"]
tf_idf_result["fake"] = tf_idf_df["fake"] * tf_idf_df["IDF"]
tf_idf_result["hate"] = tf_idf_df["hate"] * tf_idf_df["IDF"]
tf_idf_result["jsci"] = tf_idf_df["jsci"] * tf_idf_df["IDF"]
tf_idf_result["sati"] = tf_idf_df["sati"] * tf_idf_df["IDF"]
tf_idf_result["stat"] = tf_idf_df["stat"] * tf_idf_df["IDF"]

In [25]:
tf_idf_result

,word,bias,cons,fake,hate,jsci,sati,stat,IDF
0,print,0.422424,0.072532,0.000000,0.035218,0.016987,0.023736,0.000000,0.866359
1,pay,0.180876,0.207049,0.234293,0.228007,0.130928,0.048784,0.066222,0.890312
2,back,0.279246,0.269566,0.256333,0.368244,0.329463,0.160120,0.128802,0.487032
3,money,0.270700,0.260536,0.000000,0.416923,0.185635,0.037826,0.071722,0.788947
4,plu,0.051928,0.120370,0.000000,0.070134,0.183243,0.009848,0.023764,1.437751
5,interest,0.131766,0.288902,0.118162,0.377223,0.088042,0.025629,0.030924,0.748361
6,entir,0.183055,0.131192,0.000000,0.197072,0.190116,0.090560,0.014569,0.881448
7,famili,0.252850,0.298700,0.039303,0.327841,0.226953,0.122753,0.141944,0.746749
8,everyon,0.163461,0.132949,0.100295,0.255630,0.093412,0.104417,0.015749,0.952804
9,came,0.198116,0.143232,0.081039,0.203422,0.090573,0.068550,0.120889,0.769873


In [27]:
tf_idf_result.sample(10)

,word,bias,cons,fake,hate,jsci,sati,stat,IDF
3207,traffick,0.009833,0.020262,0.114638,0.044271,0.000000,0.000000,0.000000,2.178113
3044,brother,0.101126,0.065115,0.000000,0.216254,0.013725,0.086299,0.023140,1.399962
19205,framingham,0.000000,0.000000,0.000000,0.000000,0.031158,0.000000,0.000000,3.178113
7122,hilal,0.023260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.576053
15666,truest,0.000000,0.000000,0.000000,0.012919,0.000000,0.000000,0.000000,3.178113
13854,etch,0.000000,0.000000,0.000000,0.012919,0.000000,0.000000,0.000000,3.178113
19358,roposo,0.000000,0.000000,0.000000,0.000000,0.062316,0.000000,0.000000,3.178113
14015,uncompromisingli,0.000000,0.000000,0.000000,0.012919,0.000000,0.000000,0.000000,3.178113
20424,populaceno,0.000000,0.000000,0.000000,0.000000,0.031158,0.000000,0.000000,3.178113
21540,mustard,0.000000,0.000000,0.000000,0.000000,0.000000,0.111000,0.044644,2.700992
